In [1]:

# %load_ext autoreload
# %autoreload 2

# Update Notes: 

### V2: 
do not filter performance data for active section. <br>
do not round for 5 year target percentage calculation

## TODO
needs to change the FY related number to dynamic number

# Tip for quick search

* Needs attention: the place where needs update or better logic
* question to be answered: the place where things are still not clear
* Manual Check: Unit test where you can drill in to find the data that leads to the check results for a specific project and specific check
* TODO: things needs to be done
* bookmark: stop point from last visit


# Admin Notes:


1. The AMTool dataset is archived daily as csv files and used for the project book check. 
The csv files are located at: 
r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'

2. The excel input files are checked daily and archived with datestamp whenever it is modified.
The continuously updated excel input files are located at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\excel'
The excel input file are archived at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\Data_MiscInput'
To recover the archived excel file used in project book check for a target date, select the excel file with latest datestamp but is still earlier than the target date.

3. The check summary export action is logged daily. It can be used for daily monitoring. 
The file export log is located at: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal\log

4. The published data are at:

    * csv files for district asset manager: http://svgcshopp.dot.ca.gov/DataLake/ProjectBookCheck/
    * csv files for HQ AM: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal
    * tableau workbook with live data source: https://tableau.dot.ca.gov/#/site/AssetManagement/workbooks/1815/views


<a id='TableOfContents'></a>

# Table Of Contents

## Data Preprocessing

### [Global Constants](#GlobalConstants)


### [Load and cleanup source data](#Read_Data)

* [Counties](#Counties)
* [Programming_Summary](#Programming_Summary)
* [ProgrammingList](#ProgrammingList)
* [SHOPP_Raw_Data](#SHOPP_Raw_Data)
* [TenYrShopp_Perf_RawData](#TenYrShopp_Perf_RawData)


## Add fields to SHOPP raw data (calculate and join)
* [Calculated Fields](#AddDataColumns)
* [Join Tables](#DataJoining)




## [Export Projectbook Check Sumary Key Dates](#Export_KeyDates)



## [Final Clean Up](#FinalCleanUp)


# Import common modules

In [2]:

from datetime import datetime
import os.path

# import requests
import pandas as pd

import numpy as np
import re

import shutil

In [3]:
from itertools import product

In [4]:
import time
start_time = time.time()

In [5]:
#show dataframe without skip column
pd.options.display.max_columns = 100

In [6]:
# from config_datasource import *
from projectbookcheck_utilityfunction import *

You are using the Extract API 2.0, please save the output as .hyper format


# Data clean process

* funding amount: remove dollar sign, 
* fill missing value, string, numerical, 
* remove leading single quote for string value
* strip off leading and trailing space 

* regulate column names




<a id='GlobalConstants'></a>
## Global Constants

In [7]:
from constants import *

In [8]:
TARGETDATE = datetime.today().strftime("%m-%d-%Y")
CURRENT_FY = fiscalyear (datetime.today())

In [9]:
# file_export_log = open(LOG_FILE, "a")  # append mode
# file_export_log.write("#####{} \n".format(TARGETDATE))

<a id='Read_Data'></a>

# Read Data


In [10]:
if DATA_SOURCE_TYPE == 'csv':
    filename = 'TenYrShopp_RawData_'
    df_SHOPP_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, TARGETDATE))

    filename = 'TenYrShopp_PerfM_Raw_Data_'
    df_perf_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, TARGETDATE))

else:
    print('skip getting csv data.')

C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Done with TenYrShopp_RawData_ in -13.425710678100586
Done with TenYrShopp_PerfM_Raw_Data_ in -96.45661997795105
Done with Rawdata_Pavement_Worksheet_ in -8.857773303985596
Done with Rawdata_Drainage_Worksheet_ in -43.5689959526062
Done with Rawdata_Bridge_Worksheet_ in -7.964364290237427
Done with Rawdata_TMS_Worksheet_ in -9.304267168045044
Done with Project_Postmile_Check_ in -12.754770517349243
Done with Programming_Summary_ in -15.230461597442627
Done with HM_Project_Details_Raw_Data_ in -7.1786627769470215
Done with Minor_Project_Details_Raw_Data_ in -7.667008399963379
total time: -222.40863466262817

<a id='SHOPP_Raw_Data'></a>
## SHOPP Raw Data


In [11]:
df_SHOPP_raw_data.name = 'df_SHOPP_raw_data'

In [12]:
#rename columns 
dict_rename_rawdata = {
                       'County':'County TYP',
                       'Route': 'Route TYP',
                       'BackPM':'BackPM TYP',
                       'AheadPM':'AheadPM TYP',
                       'ID': 'AMT_ID',
                       'Ten-Year Plan': 'Ten-Year Plan RD',
                       'County.1' : 'County PRG',
                       'Route.1' : 'Route PRG',
                       'BackPM.1':'BackPM PRG',
                       'AheadPM.1' : 'AheadPM PRG',
                       'County.2' : 'County PCR',
                       'Route.2' : 'Route PCR',
                       'BackPM.2':'BackPM PCR',
                       'AheadPM.2' : 'AheadPM PCR',
                       'Activity Category': 'Activity'
                      }

df_SHOPP_raw_data = df_SHOPP_raw_data.rename(dict_rename_rawdata, axis = 1)

In [13]:
#remove leading puncture for target columns
cols_strip = ['District','Route TYP','EA','EFIS','Route PRG','PPNO','Route PCR']
for c in cols_strip :
    df_SHOPP_raw_data[c] = df_SHOPP_raw_data[c].str.strip("'")

In [14]:
cost_columns = [
    'RW Cost ($K)',
    'Const Cost ($K)',
    'Support Cost ($)',
    'TYP Total Project Cost ($K)',
    'PAED ($K)',
    'PSE ($K)',
    'R/W ($K)',
    'CONS ($K)',
    'Prog Support Cost ($)',
    'Prog RW Cost ($K)',
    'Prog Const Cost ($K)',
    'Prog Total Project Cost ($K)',
    'PCR R/W Cap ($K)',
    'PCR Const Cap ($K)',
    'PCR Support Cost ($K)',
    'PCR Total Cost ($K)',
    'Project Cost In Use',
    'Total LL Prog ($K)',
    'LL PAED Cost ($K)',
    'LL CONS Cap ($K)',
    'PA&ED Cost',
    'LL Adl RW ($K)',
    'LL PSE ($K)',
    'PAED ($K)',  
    'LL CONS ($K)',
    'LL RW Cap ($K)',
               ]

for c in cost_columns:
    df_SHOPP_raw_data[c] = df_SHOPP_raw_data[c].apply(curreny_to_float)
    df_SHOPP_raw_data[c].fillna(0, inplace = True)

In [15]:
#data clean 
#data type regulation
#string/text data regulation

df_SHOPP_raw_data['District'] =df_SHOPP_raw_data['District'].astype(int)

df_SHOPP_raw_data['EFIS'] = pd.to_numeric(df_SHOPP_raw_data['EFIS'], errors='coerce')

df_SHOPP_raw_data['Route PCR'] = df_SHOPP_raw_data['Route PCR'].astype(str)

#data trimming
#row trimming
df_SHOPP_raw_data= df_SHOPP_raw_data[df_SHOPP_raw_data['District'] != 56]

#column trimming
df_SHOPP_raw_data.drop(['District Priority', 'PIR Performance Report'],
  axis='columns', inplace=True, errors='ignore')


#Question to be answered:
#for EA Raw Data, the missing data is not null , but ''. Should we handle the missing data uniformly for string data?

#TODO:
#fill missing data
#data quality check (checksum,)

In [16]:
df_SHOPP_raw_data['CCA Date Miilestone (M600)'] = df_SHOPP_raw_data['CCA Date Miilestone (M600)'].apply(regulate_timestamp_format)

In [17]:
# df_SHOPP_raw_data.shape

<a id='TenYrShopp_Perf_RawData'></a>
## TenYrShopp_Perf_RawData


In [18]:
#rename columns
dict_rename_perf_rawdata = {
                           'ID': 'AMT_ID',
              }

df_perf_raw_data = df_perf_raw_data.rename(dict_rename_perf_rawdata, axis = 1)

In [19]:
cols_strip = ['EA','EFIS','PPNO']
for c in cols_strip :
    df_perf_raw_data[c] = df_perf_raw_data[c].str.strip("'")

In [20]:
#data clean 
#data type regulation

df_perf_raw_data['Quantity'] = df_perf_raw_data['Quantity'].fillna(0)
df_perf_raw_data['Assets in Good Cond'] = df_perf_raw_data['Assets in Good Cond'].fillna(0)
df_perf_raw_data['Assets in Fair Cond'] = df_perf_raw_data['Assets in Fair Cond'].fillna(0)
df_perf_raw_data['Assets in Poor Cond'] = df_perf_raw_data['Assets in Poor Cond'].fillna(0)
df_perf_raw_data['New Assets Added'] = df_perf_raw_data['New Assets Added'].fillna(0)

df_perf_raw_data['EFIS'] = pd.to_numeric(df_perf_raw_data['EFIS'], errors='coerce')


In [21]:
#data trimming
#row
df_perf_raw_data= df_perf_raw_data[df_perf_raw_data['District'] != 56]
#column
df_perf_raw_data.drop(['PID Cycle', 'TYP','ProjectedSHOPP Cycle','RequestedRTL FY','DistrictPriority'],
  axis='columns', inplace=True, errors='ignore')

In [22]:
df_perf_raw_data.name = 'df_perf_raw_data'

<a id='FinanceTargetsAndUnitCost'></a>
## FinanceTargetsAndUnitCost


In [23]:
# Finance Targets

filename = 'Targets and Unit cost.xlsx'

shts = ['Targets and Unit Cost', 'Finance Targets']

dict_df = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), sheet_name = shts) 

df_finance_targets_uc = dict_df['Targets and Unit Cost']
df_finance_targets = dict_df['Finance Targets']

df_finance_targets.name = 'df_finance_targets'
df_finance_targets_uc.name = 'df_finance_targets_uc'


In [24]:

#rename columns
# dict_rename= {
#                'Objective':'Performance Objective', 
#     'Unit': 'Unit of Measure',
#     'New Performance Target': 'New Target',
#     'Fair to Good': 'F2G Target',
#     'Poor to Good': 'P2G Target',
#               }

# df_finance_targets_uc = df_finance_targets_uc.rename(dict_rename, axis = 1)


# df_factflow_financetarget = df_perf_raw_prog_county.merge(df_finance_targets, how = 'left', 
#                   left_on = 'District', 
#                   right_on = 'District Targets')



<a id='ProgrammingList'></a>
## Programming List


In [25]:
shts = ['2010 SHOPP',
        '2012 SHOPP',
        '2014 SHOPP',
        '2016 SHOPP',
        '2018 SHOPP',
        '2020 SHOPP',
        'Long Lead'
        ]

filename = 'Programming_list.xlsx'

df_dict = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), sheet_name =shts) 


df_program = pd.DataFrame()

for k, v in df_dict.items():
#     print(type(v))
#     print(k)
    v['Table Names'] = k
#     print(v.columns)
    df_program = df_program.append(v)

In [26]:
#rename columns
dict_rename_program = {
#                         'EA':'EA', 
                        'EFIS':'EFIS_Program', 
#                         'PPNO':'PPNO Programming', 
                        'Total Capital & Support':'Total Capital & Support Cost',
#                         'Route': 'Route Programming',
              }

df_program = df_program.rename(dict_rename_program, axis = 1)

In [27]:
#data clean 
#data type regulation
# df_program['Dist'] = df_program['Dist'].astype(int)
# df_program['EFIS'] = pd.to_numeric(df_program['EFIS'], errors='coerce')

fillna_columns = ['Con Sup','RW Sup','PA&ED','PS&E', 'PA&ED', 'RW', 'Con']

df_program[fillna_columns].fillna(0, inplace=True)
df_program['Route'].fillna('Various', inplace=True)

df_program['Support Cost'] = df_program['Con Sup']+df_program['RW Sup']+df_program['PA&ED']+df_program['PS&E']
df_program['Capital Cost'] = df_program['Con']+df_program['RW']


C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\pandas\core\frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [28]:
#data augmentation
#data transformation

# df_program.dropna(subset = ['EFIS_Program'], inplace = True)
df_program['FY'] = df_program['FY'].apply(FY_cleanup)

df_program['Begin Post Miles'] = df_program['Post Miles'].apply(lambda x: str(x).split('/')[0])
df_program['End Post Miles'] = df_program['Post Miles'].apply(lambda x: str(x).split('/')[0] if '/' in str(x) else np.NaN)

In [29]:
#data trimming
df_program.drop(['PM1BF', 'PM1B', 'PM1AF','PM1A',], axis='columns', inplace=True)

In [30]:
df_program.name = 'df_program'

In [31]:
#data sanity check
#check duplicates
#check null
#check data type

if df_program['EFIS_Program'].value_counts(dropna = False).max() > 1:
    Print('Duplicate EFIS ID found, please check the source data')

<a id='HOV_Degradation'></a>
## HOV Degradation

In [32]:
filename = 'HOV Degradation.xlsx'

df_HOV_degradataion = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename))

# Add columns to raw data

In [33]:
df_SHOPP_raw_data['Data_TimeStamp'] = TARGETDATE

In [34]:
# is reservation project?
reservation_list=[
    'Major Damage - Emergency Opening',
    'Major Damage - Permanent Restoration',
    'Reactive Safety',
    'Safety - Monitoring',
    'Safety - SI',
    'Relinquishment',
    'Bridge - Deck',
    ]

def ck_is_reservation(df):
    if df['Activity'] in reservation_list:
        return 'Yes'
    else:
        return 'No'
        
col_name =  'Reservation Project?'
df_SHOPP_raw_data[col_name] = df_SHOPP_raw_data.apply(ck_is_reservation, axis = 1)

In [35]:
# Add 'FY','Total Capital & Support Cost' to raw data
df_SHOPP_raw_data.drop(columns =['EFIS_Program','FY','Total Capital & Support Cost','Capital Cost', 'Support Cost'], inplace=True, errors = 'ignore')
df_SHOPP_raw_data = pd.merge(df_SHOPP_raw_data, df_program[['EFIS_Program','FY','Total Capital & Support Cost','Capital Cost', 'Support Cost' ]], 
                             how = 'left', left_on = 'EFIS', right_on='EFIS_Program')

In [36]:
#add col 'Include 5-year POR?'
df_SHOPP_raw_data['Section'] = df_SHOPP_raw_data.apply(cal_section_in_use, axis=1)     

df_SHOPP_raw_data['AM Tool RTL (Section in Use)'] = df_SHOPP_raw_data.apply(calc_SIU_RTL, axis=1)

df_SHOPP_raw_data['AM Tool RTL (Section in Use)'].fillna('00', inplace = True)

df_SHOPP_raw_data['Activity (group)'] = df_SHOPP_raw_data['Activity'].apply(calc_activity_group)

df_SHOPP_raw_data['Last Year FY POR'] = df_SHOPP_raw_data['AM Tool RTL (Section in Use)'].str[-2:].astype(int)+2000


def calc_include_5year_POR(df):
    if df['Ten-Year Plan RD'] == 9999:
        return  'No'
    elif(df['Last Year FY POR']>TARGET_FY and df['Last Year FY POR']<TARGET_FY + 6) :   
        if df['Activity (group)'] == 'Reservation' and pd.isnull(df['SHOPP Amendment Date']): 
            return  'Yes' 
        else: 
            return 'No' 
    elif (df['Last Year FY POR']>TARGET_FY + 5 and df['Last Year FY POR']<TARGET_FY + 11) : 
        if(df['Long Lead'] == "Y") and (df['Section'] == "PRG")  :
             return  'Yes'
        elif pd.isnull(df['SHOPP Amendment Date']): 
             return  'Yes' 
        else: 
            return 'No' 
    else: 
        return 'No'

df_SHOPP_raw_data['Include 5-year POR?'] = df_SHOPP_raw_data.apply(calc_include_5year_POR, axis = 1)

In [37]:
def calc_paed_allocation(df):
    if pd.notnull(df['SHOPP Amendment Date']) or df['Include 5-year POR?'] == 'No' or df['Reservation Project?'] == 'Yes':
        return 0
    else:
        if pd.isnull(df['PA&ED Cost']):
            return 0
        else:
            return float(df['PA&ED Cost'])

df_SHOPP_raw_data['PA&ED Allocation'] =  df_SHOPP_raw_data.apply(calc_paed_allocation, axis = 1)


In [38]:
def calc_paed_allocation_year(df):
    if df['Long Lead'] != 'Y':
        return np.nan
    elif pd.notnull(df['SHOPP Amendment Date']):
        return np.nan
    elif df['Section In Use'] =='TYP':
        return df['Target RTL FY']
    else:
        return df['Requested RTL FY']
    
df_SHOPP_raw_data['PA&ED allocation Year'] = df_SHOPP_raw_data.apply(calc_paed_allocation_year, axis = 1)


In [39]:
def calc_por_paed_year(df):
    if pd.isnull(df['PA&ED allocation Year'] ):
        return np.nan
    
    int_FY = int(df['PA&ED allocation Year'][-2:])
    if int_FY < 27: 
        return 'LL PA&ED'
    else:
        return df['PA&ED allocation Year']
df_SHOPP_raw_data['POR PA&ED Year'] = df_SHOPP_raw_data.apply(calc_por_paed_year, axis = 1)

In [40]:
def calc_projectcost_POR_RTL(df):
    if df['Reservation Project?'] == 'Yes':
        return 0
    elif df['Include 5-year POR?'] =='Yes':
        return df['Project Cost In Use'] - df['PA&ED Cost']
    else:
        return 0
    
df_SHOPP_raw_data['Project Cost in RTL Year of POR'] = df_SHOPP_raw_data.apply(calc_projectcost_POR_RTL, axis = 1)


In [41]:
def calc_support_cost(df):
    if pd.isnull(df['Long Lead']): #if it is not an active long lead project
        if df['Section']=='TYP':
            return df['Support Cost ($)'] 
        elif df['Section']=='PRG':
            return df['Prog Support Cost ($)']
        elif df['Section']=='PPC':
            return df['PCR Support Cost ($K)']
        elif df['Section']=='CCA': # it is in CCA now
            if pd.isnull(df['PCR SHOPP Amendment Date']):   #no information in PPC section to be used
                return df['Prog Support Cost ($)']
            else: #use the PPC section information 
                return df['PCR Support Cost ($K)']
        
    else: #for a long lead project
        if df['Section']=='TYP':
            return df['Support Cost ($)'] + df['LL PAED Cost ($K)']
        elif df['Section']=='PRG':
            return df['LL Adl RW ($K)'] + df['LL PSE ($K)'] + df['PAED ($K)'] + df['LL CONS ($K)']
        elif df['Section']=='PPC': 
            return df['PCR Support Cost ($K)']
        elif df['Section']=='CCA': #no information in PPC section to be used
            if pd.isnull(df['PCR SHOPP Amendment Date']):
                return df['LL Adl RW ($K)'] + df['LL PSE ($K)'] + df['PAED ($K)'] + df['LL CONS ($K)']
            else:
                return df['PCR Support Cost ($K)']   

                
def calc_capital_cost(df):
    if pd.isnull(df['Long Lead']): #if it is not an active long lead project
        if df['Section']=='TYP':
            return df['RW Cost ($K)'] + df['Const Cost ($K)']
        elif df['Section']=='PRG':
            return df['Prog RW Cost ($K)'] + df['Prog Const Cost ($K)']
        elif df['Section']=='PPC':
            return df['PCR R/W Cap ($K)'] + df['PCR Const Cap ($K)']
        elif df['Section']=='CCA': # it is in CCA now
            if pd.isnull(df['PCR SHOPP Amendment Date']):   #no information in PPC section to be used
                return df['Prog RW Cost ($K)'] + df['Prog Const Cost ($K)']
            else: #use the PPC section information 
                return df['PCR R/W Cap ($K)'] + df['PCR Const Cap ($K)']
        
    else: #for a long lead project
        if df['Section']=='TYP':
            return df['RW Cost ($K)'] + df['Const Cost ($K)']
        elif df['Section']=='PRG':
            return df['LL RW Cap ($K)'] + df['LL CONS Cap ($K)']
        elif df['Section']=='PPC': 
            return df['PCR R/W Cap ($K)'] + df['PCR Const Cap ($K)']
        elif df['Section']=='CCA': #no information in PPC section to be used
            if pd.isnull(df['PCR SHOPP Amendment Date']):
                return df['LL RW Cap ($K)'] + df['LL CONS Cap ($K)']
            else:
                return df['PCR R/W Cap ($K)'] + df['PCR Const Cap ($K)']

col_name =  'Support Cost ($K)'
df_SHOPP_raw_data[col_name] = df_SHOPP_raw_data.apply(calc_support_cost, axis = 1)

col_name =  'Capital Cost ($K)'
df_SHOPP_raw_data[col_name] = df_SHOPP_raw_data.apply(calc_capital_cost, axis = 1)

In [150]:

def calc_support_capital_ratio(df):
    if df['Capital Cost ($K)'] != 0: 
        return round(df['Support Cost ($K)']/df['Capital Cost ($K)'],3)
    else:
        return 0
    
df_SHOPP_raw_data['Support Capital Ratio'] = df_SHOPP_raw_data.apply(calc_support_capital_ratio, axis = 1)

In [43]:
# df_SHOPP_raw_data['Support Capital Ratio'].value_counts(dropna = False)

In [44]:
# calculate column: HOV Degradation Project?

temp = df_perf_raw_data[df_perf_raw_data['Activity Detail'] == 'HOV Degradation Mitigation'].groupby(['AMT_ID', 'Section']).agg('first').reset_index()

temp['HOV Degradation Project?'] = 'Yes'

df_SHOPP_raw_data = pd.merge(df_SHOPP_raw_data, temp[['AMT_ID', 'Section','HOV Degradation Project?']], 
                             how = 'left', left_on = ['AMT_ID', 'Section'], right_on=['AMT_ID', 'Section'])

df_SHOPP_raw_data['HOV Degradation Project?'].fillna('No', inplace = True)

In [45]:
# check raw data calculation

# df_SHOPP_raw_data[['AMT_ID','Section','Reservation Project?','Include 5-year POR?',
#        'PA&ED Allocation', 'Project Cost in RTL Year of POR',  'PA&ED allocation Year', 'POR PA&ED Year',
#        'Support Cost ($K)',
#        'Capital Cost ($K)', 'HOV Degradation Project?'] ].to_csv('raw_data.csv', index= False)

In [46]:
#debug only
# df_SHOPP_raw_data[(df_SHOPP_raw_data['Include 5-year POR?'] == 'Yes')
#                  & (df_SHOPP_raw_data['District'] == 1)
#                  ].groupby(['District','RTL In Use']).agg(
#     {'Project Cost in RTL Year of POR': 'sum',
#     'PA&ED Allocation': 'sum',
#     'AMT_ID': 'nunique',
#     }
# ).reset_index()

# Add columns to performance data

In [47]:
#legacy problem, keep the performance of the unused sections, to compare with mara's spreadsheet.

df_perf_raw_data_1 = pd.merge(df_perf_raw_data,
                              df_SHOPP_raw_data[['AMT_ID', 'Section', 'Reservation Project?', 'Include 5-year POR?','RTL In Use','Support Capital Ratio',
                  'Project Cost In Use',]], 
                how ='left', 
                left_on = ['AMT_ID', 'Section'],
                right_on = ['AMT_ID', 'Section'], 
#                 suffixes=('', '_raw'),
                             )

In [48]:
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, df_finance_targets_uc[['District', 'Performance Objective', 
                                                                         'New Target','F2G Target','P2G Target',
                                                                         'New Escalated UC ($K)', 'F2G Escalated UC ($K)', 'P2G Escalated UC ($K)']], 
                how ='left', 
                left_on = ['District','Performance Objective'],
                right_on = ['District','Performance Objective'],)


In [49]:
df_perf_raw_data_1['Post-Fair'].fillna(0, inplace = True)

df_perf_raw_data_1['F2G Achieved'] = df_perf_raw_data_1['Post-Fair'] - df_perf_raw_data_1['Assets in Fair Cond']

In [50]:
df_perf_raw_data_1['Post-Poor'].fillna(0, inplace = True)

df_perf_raw_data_1['P2G Achieved'] = df_perf_raw_data_1['Post-Poor'] - df_perf_raw_data_1['Assets in Poor Cond']

In [51]:
dict_rename_performance ={
    'New Assets Added':'New Achieved',
}

df_perf_raw_data_1 = df_perf_raw_data_1.rename(dict_rename_performance, axis = 1)


In [52]:
df_perf_raw_data_1['Concatenate District+Objective'] = df_perf_raw_data_1['District'].astype(str) + df_perf_raw_data_1['Performance Objective']
df_perf_raw_data_1['Include in Pivot table?'] =''

In [53]:
def calc_earned_value(df):
    
    EV = (
        df['New Achieved'] * df['New Escalated UC ($K)'] 
        + df['Post-Fair'] + df['Assets in Fair Cond']* df['F2G Escalated UC ($K)']
        + df['Post-Poor'] + df['Assets in Poor Cond']* df['P2G Escalated UC ($K)']
         )
    
    return round(EV,2)

df_perf_raw_data_1['Earned Value (EV) ($K)'] = df_perf_raw_data_1.apply(calc_earned_value, axis = 1)

In [54]:
def calc_effective_earned_value(df):
    # I do not understand this function
    if df['F2G Target'] == 0: 
        EEV = 0
    else: 
        EEV = df['Post-Fair'] + df['Assets in Fair Cond'] * df['F2G Escalated UC ($K)']
    
    if df['P2G Escalated UC ($K)'] != 0 :
        EEV += df['Post-Poor'] + df['Assets in Poor Cond'] * df['P2G Escalated UC ($K)']
    
    return round(EEV,2)

df_perf_raw_data_1['Effective Earned Value (EEV) ($K)'] = df_perf_raw_data_1.apply(calc_effective_earned_value, axis = 1)

In [55]:
# df_perf_raw_data_1.columns

In [56]:
# keep the relavent performance data only

df_perf_raw_data_filtered = df_perf_raw_data_1[
    (df_perf_raw_data_1['Include 5-year POR?'] == 'Yes')
    & (df_perf_raw_data_1['Reservation Project?'] == 'No')

]

In [57]:
# df_perf_raw_data_1[
#     (df_perf_raw_data_1['Include 5-year POR?'] == 'Yes')]['Support Capital Ratio'].value_counts(dropna = False)

In [58]:
# df_perf_raw_data_1.shape

In [59]:
# #PPI Debug

# df_perf_raw_data_filtered[(df_perf_raw_data_filtered['District'] == 1)
#                  & (df_perf_raw_data_filtered['RTL In Use'] == '2026/27')
#                   & (df_perf_raw_data_filtered['Performance Objective'] == 'Proactive Safety')]

# create summary data

In [60]:

Districts =  list(range(1, 13))
RTLs = ['2026/27', '2027/28', '2028/29','2029/30', '2030/31']

df_5y_summary = pd.DataFrame(list(product(Districts, RTLs)), columns=['District', 'RTL In Use'])

In [61]:
#calculate total project cost for each district and RTL year
temp = df_SHOPP_raw_data[(df_SHOPP_raw_data['Include 5-year POR?'] =='Yes')]
temp1 = temp.groupby(['District','RTL In Use'])['Project Cost in RTL Year of POR'].agg(sum).reset_index()

df_5y_summary = pd.merge(df_5y_summary, temp1, 
                             how = 'left', left_on = ['District','RTL In Use'], right_on=['District','RTL In Use'])

In [62]:
df_5y_summary['Project Cost in RTL Year of POR'].fillna(0, inplace = True)

In [63]:
# calculate the PAED allocation for each RTL year 
temp = df_SHOPP_raw_data[(df_SHOPP_raw_data['Include 5-year POR?'] =='Yes') ]

df_PAED= temp.groupby(['District','POR PA&ED Year']).agg(
    {
    'PA&ED Allocation': 'sum',
    }
 ).reset_index()

In [64]:

df_5y_summary = pd.merge(df_5y_summary, df_PAED, 
                             how = 'left', left_on = ['District','RTL In Use'], right_on=['District','POR PA&ED Year'])
df_5y_summary['PA&ED Allocation'].fillna(0, inplace = True)


df_5y_summary['Project Cost ($K)'] = df_5y_summary['Project Cost in RTL Year of POR'] +  df_5y_summary['PA&ED Allocation']

In [65]:
# find funding target for each district and RTL year
df_5y_summary = pd.merge(df_5y_summary, df_finance_targets[['District', '5-Y Funding Targets ($K)']], 
                             how = 'left', left_on = 'District', right_on='District')

In [66]:
df_5y_summary['Annual Target Funding ($K)'] = df_5y_summary['5-Y Funding Targets ($K)']/5
df_5y_summary['5% Upper Limit ($M)'] = df_5y_summary['5-Y Funding Targets ($K)']*0.25/1000
df_5y_summary['5% Lower Limit ($M)'] = df_5y_summary['5-Y Funding Targets ($K)']*0.15/1000

In [67]:
#count number of non-reservation projects within 5 year POR, for each district and RTL year.

temp1 = df_SHOPP_raw_data[
    (df_SHOPP_raw_data['Reservation Project?'] =='No') 
    & (df_SHOPP_raw_data['Include 5-year POR?'] =='Yes')]

temp2 = temp1.groupby(['District','RTL In Use'])['AMT_ID'].agg('count').reset_index(name = 'No of Projects')

df_5y_summary = pd.merge(df_5y_summary, temp2, 
                             how = 'left', left_on = ['District','RTL In Use'], right_on=['District','RTL In Use'])

df_5y_summary['No of Projects'].fillna(0, inplace = True)

In [68]:
df_5y_summary['Balance ($K)'] = round(df_5y_summary['Annual Target Funding ($K)']-df_5y_summary['Project Cost ($K)'],0)

df_5y_summary['Percent of 5-Year Target Funding'] = df_5y_summary['Project Cost ($K)'] / df_5y_summary['5-Y Funding Targets ($K)']

df_5y_summary['Within 20% (+/-5%) Annual Target Funding?'] = df_5y_summary['Percent of 5-Year Target Funding'].apply(lambda x: 1 if (x < 0.15 or x > 0.25) else 0)

In [69]:
# df_5y_summary

## calculate PPI

In [70]:
Districts =  list(range(1, 13))
RTLs = ['2026/27', '2027/28', '2028/29','2029/30', '2030/31']

df_PPI = pd.DataFrame(list(product(Districts, RTLs)), columns=['District', 'RTL In Use'])

In [71]:
# sum project cost for projects within 5y POR, for each district and RTL

temp = df_SHOPP_raw_data[df_SHOPP_raw_data['Include 5-year POR?']=='Yes'].groupby(['District','RTL In Use'])['Project Cost in RTL Year of POR'].agg(sum).reset_index(name = 'Project Cost')

df_PPI = pd.merge(df_PPI, temp, how = 'left', left_on = ['District','RTL In Use'], right_on = ['District','RTL In Use'])

In [72]:
# sum PA&ED Allocation cost for projects within 5y POR, for each district and RTL

temp = df_SHOPP_raw_data[df_SHOPP_raw_data['Include 5-year POR?']=='Yes'].groupby(['District','POR PA&ED Year'])['PA&ED Allocation'].agg(sum).reset_index(name = 'Total PAED Allocation')

df_PPI = pd.merge(df_PPI, temp, how = 'left', left_on = ['District','RTL In Use'], right_on = ['District','POR PA&ED Year'])

In [73]:
df_PPI.fillna(0, inplace =True)

df_PPI['Total Project Cost'] = df_PPI['Project Cost'] + df_PPI['Total PAED Allocation']

In [74]:
# augment performance objective target and unit cost for each district and RTL

df_PPI_performace= pd.merge(df_PPI[['District','RTL In Use']],
                            df_finance_targets_uc[[
                                'District', 'Performance Objective',
                                'New Target', 'F2G Target', 'P2G Target',
                                'New Escalated UC ($K)',   'F2G Escalated UC ($K)', 'P2G Escalated UC ($K)',
                            ]],
                            how = 'left', left_on = ['District'], right_on = ['District'])
# df_PPI

In [75]:
# calculate achieved performance for each district, RTL and performance objective

temp = df_perf_raw_data_filtered.groupby(['District','RTL In Use','Performance Objective'])[['New Achieved','F2G Achieved', 'P2G Achieved']].sum().reset_index()

df_PPI_performace = pd.merge(df_PPI_performace, temp, how = 'left', left_on= ['District','RTL In Use','Performance Objective'], right_on= ['District','RTL In Use','Performance Objective'])


In [76]:
df_PPI_performace['New Achieved'].fillna(0, inplace = True)
df_PPI_performace['F2G Achieved'].fillna(0, inplace = True)
df_PPI_performace['P2G Achieved'].fillna(0, inplace = True)

In [77]:
df_PPI_performace['New Achieved']=-abs(df_PPI_performace['New Achieved'])

In [78]:
#calculate cumulative achieved performance 

# temp = df_PPI_performace.groupby(['Performance Objective','District','RTL In Use']).sum().groupby(level=[0,1]).cumsum().reset_index()
temp = df_PPI_performace.groupby(['Performance Objective','District','RTL In Use']).sum()[['New Achieved','F2G Achieved', 'P2G Achieved']].groupby(['Performance Objective','District']).cumsum().reset_index()

temp1 = pd.merge(df_PPI_performace,
                             temp[['Performance Objective','District','RTL In Use','New Achieved','F2G Achieved', 'P2G Achieved']],
                             how = 'left', 
                             left_on= ['District','RTL In Use','Performance Objective'], 
                             right_on= ['District','RTL In Use','Performance Objective'],
                            suffixes = ['', ' Cumulated'])

df_PPI_performace = temp1.sort_values(['District','Performance Objective','RTL In Use'])


In [79]:
# calculate earned value for each district, RTL

def calc_earned_value(escalated_uc, cumulated_achieved_performance, target_performance ):
    return round(escalated_uc * min(abs(cumulated_achieved_performance), target_performance), 3)
    
# cumulative_earned_performance= unitcost*(1+escalationrate)^year*min(achieved_performance, target_performance)
# current_RTL_earned_performance = difference in cumulative_earned_performance

df_PPI_performace['New Earned Value'] = df_PPI_performace.apply(lambda x: calc_earned_value(x['New Escalated UC ($K)'], x['New Achieved Cumulated'],x['New Target']), axis = 1)
df_PPI_performace['F2G Earned Value'] = df_PPI_performace.apply(lambda x: calc_earned_value(x['F2G Escalated UC ($K)'], x['F2G Achieved Cumulated'],x['F2G Target']), axis = 1)
df_PPI_performace['P2G Earned Value'] = df_PPI_performace.apply(lambda x: calc_earned_value(x['P2G Escalated UC ($K)'], x['P2G Achieved Cumulated'],x['P2G Target']), axis = 1)

In [80]:
#debug only

# df_PPI_performace[df_PPI_performace['Performance Objective']== 'Proactive Safety'][['District', 'RTL In Use', 'Performance Objective','New Target',
#        'F2G Target', 'P2G Target','New Achieved', 'F2G Achieved', 'P2G Achieved','New Achieved Cumulated', 'F2G Achieved Cumulated',
#        'P2G Achieved Cumulated','New Escalated UC ($K)', 'F2G Escalated UC ($K)',
#        'P2G Escalated UC ($K)','New Earned Value',
#        'F2G Earned Value', 'P2G Earned Value']]

In [81]:
#calculate cumulative earned value of all performance objectives for each RTL
#sum all the cumulative earned value from NEW, F2G and P2G to total value

temp = df_PPI_performace.groupby(
    ['District','RTL In Use',]
)[['New Earned Value','F2G Earned Value', 'P2G Earned Value']].sum().reset_index()

temp['Cumulative Total Value'] = temp['New Earned Value'] + temp['F2G Earned Value'] + temp['P2G Earned Value']

In [82]:
#augment a row ahead of 5 year cycle, to find the difference for each of the 5 year POR

Districts =  list(range(1, 13))
RTLs = ['2025/26']
df_dummy = pd.DataFrame(list(product(Districts, RTLs)), columns=['District', 'RTL In Use'])

temp1 = pd.merge(temp, df_dummy, how ='outer', left_on=['District', 'RTL In Use'], right_on=['District', 'RTL In Use'])

temp1.fillna(0, inplace= True)

temp1.sort_values(['District', 'RTL In Use'], inplace =  True)

temp1['Total Value'] = temp1['Cumulative Total Value'].diff()

df_PPI.drop(['Total Value'], axis='columns', inplace=True, errors='ignore')

df_PPI = pd.merge(df_PPI, temp1[['District', 'RTL In Use','Total Value']], how ='left',  left_on=['District', 'RTL In Use'], right_on=['District', 'RTL In Use'])

In [83]:
df_PPI['PPI'] = round(df_PPI['Total Value']/df_PPI['Total Project Cost']* 100,0)

In [84]:
#debug only

# df_PPI_performace.to_csv('./output/PPI_performance.csv', index = False)

In [85]:
#debug only

# df_PPI_performace[(df_PPI_performace['District'] == 1)
#                  & (df_PPI_performace['RTL In Use'] == '2026/27')
# #                   & (df_PPI_performace['Performance Objective'] == 'Pavement Class II')
# #                   & (df_PPI_performace['RTL In Use'] == '2027/28')
#                  ][['District', 'RTL In Use', 'Performance Objective','New Target',
#        'F2G Target', 'P2G Target','New Achieved', 'F2G Achieved', 'P2G Achieved','New Achieved Cumulated', 'F2G Achieved Cumulated',
#        'P2G Achieved Cumulated','New Escalated UC ($K)', 'F2G Escalated UC ($K)',
#        'P2G Escalated UC ($K)','New Earned Value',
#        'F2G Earned Value', 'P2G Earned Value']]

In [86]:
df_5y_summary = pd.merge(df_5y_summary, df_PPI[['District','RTL In Use','PPI']],
                             how = 'left', left_on = ['District','RTL In Use'], right_on=['District','RTL In Use'])

In [87]:
df_5y_summary = df_5y_summary.sort_values(['RTL In Use', 'District'])

In [88]:
#rearrange columns sequence

cols = [
'District', 
       '5-Y Funding Targets ($K)', 'Annual Target Funding ($K)',
       '5% Upper Limit ($M)', '5% Lower Limit ($M)', 
    'RTL In Use', 'Project Cost ($K)','No of Projects',
       'Balance ($K)', 'Percent of 5-Year Target Funding',
       'Within 20% (+/-5%) Annual Target Funding?','PPI'
]
df_5y_summary = df_5y_summary[cols]

In [89]:
# df_5y_summary

# Caluclate LL summary

In [90]:
#create an empty df with all target district and RTL

Districts =  list(range(1, 13))
RTLs = ['LL PA&ED']

df_LL_summary = pd.DataFrame(list(product(Districts, RTLs)), columns=['District', 'RTL In Use'])

In [91]:
# calculate supprt and capital cost for long lead project
temp = df_SHOPP_raw_data[(df_SHOPP_raw_data['Reservation Project?'] =='No')
                  & (df_SHOPP_raw_data['Include 5-year POR?'] =='Yes')   
                        ]

temp1= temp.groupby(['District']).agg(
    {
    'Support Cost ($K)': 'sum',
    'Capital Cost ($K)': 'sum',
    }
 ).reset_index()



df_LL_summary = pd.merge(df_LL_summary, temp1, 
                             how = 'left', left_on = ['District'], right_on=['District'])

In [92]:

df_LL_summary = pd.merge(df_LL_summary, df_PAED, 
                             how = 'left', left_on = ['District','RTL In Use'], right_on=['District','POR PA&ED Year'])

In [93]:
df_LL_summary['PA&ED Allocation'].fillna(0, inplace = True)
dict_rename = {
    'PA&ED Allocation': 'Project Cost ($K)',
                               }

df_LL_summary.rename(dict_rename, axis = 1, inplace = True)

In [94]:

df_LL_summary = pd.merge(df_LL_summary, df_finance_targets[['District', '5-Y Funding Targets ($K)']], 
                             how = 'left', left_on = 'District', right_on='District')

df_LL_summary['Percent of 5-Year Target Funding'] = round(df_LL_summary['Project Cost ($K)'] / df_LL_summary['5-Y Funding Targets ($K)'],3)



In [ ]:

df_LL_summary['Support Capital Ratio'] = df_LL_summary['Support Cost ($K)']/df_LL_summary['Capital Cost ($K)']



In [95]:
#count number of LL PAED projects

temp = df_SHOPP_raw_data[(df_SHOPP_raw_data['Reservation Project?'] =='No')
                  & (df_SHOPP_raw_data['Include 5-year POR?'] =='Yes')   
                         & (df_SHOPP_raw_data['POR PA&ED Year'] =='LL PA&ED')
                        ]

temp1 = temp.groupby(['District'])['PA&ED Allocation'].agg(
    'count').reset_index(name = 'No of Projects')

df_LL_summary = pd.merge(df_LL_summary, temp1, 
                             how = 'left', left_on = 'District', right_on='District')

In [96]:

temp = df_perf_raw_data_filtered[
                   (df_perf_raw_data_filtered['Performance Objective'] =='Complete Streets Build New')
                   &  (df_perf_raw_data_filtered['RTL In Use'].isin(['2026/27','2027/28']))
                  ].groupby(['District']).agg({'P2G Achieved':'sum'}).reset_index()


df_LL_summary =  pd.merge(df_LL_summary, 
                             temp, 
                            how = 'left', 
                            left_on = ['District'], 
                  right_on = ['District'])

In [97]:
temp2 = df_finance_targets_uc[df_finance_targets_uc['Performance Objective'] == 'Complete Streets Build New'][['District', 'P2G Target']]

df_LL_summary =  pd.merge(df_LL_summary, 
                             temp2, 
                            how = 'left', 
                            left_on = ['District'], 
                  right_on = ['District'])

In [98]:
df_LL_summary['Complete Streets Build New Performance (Years 6 & 7) % of Target'] = round(-df_LL_summary['P2G Achieved']/df_LL_summary['P2G Target'],3)

In [99]:
df_LL_summary['Complete Streets Build New Performance (Years 6 & 7) Greater Than 40% Annual Performance Target?'] = df_LL_summary['Complete Streets Build New Performance (Years 6 & 7) % of Target'].apply(lambda x: -1 if x > 0.4 else 1)

In [100]:
# df_perf_raw_data_1[(df_perf_raw_data_1['Reservation Project?'] =='No')
#                   & (df_perf_raw_data_1['Include 5-year POR?'] =='Yes')
#                    &  (df_perf_raw_data_1['Performance Objective'] =='Complete Streets Build New')
#                    &  (df_perf_raw_data_1['RTL In Use'].isin(['2026/27','2027/28',]))
#                    & (df_perf_raw_data_1['District'] == 1)
#                   ]

In [101]:
# df_perf_raw_data_filtered[(df_perf_raw_data_filtered['Performance Objective'] =='Complete Streets Build New')
#                          & (df_perf_raw_data_filtered['District'] == 1)
#                           &  (df_perf_raw_data_filtered['RTL In Use'].isin(['2026/27','2027/28',]))
#                          ]

In [102]:
df_LL_summary =  pd.merge(df_LL_summary, 
                             df_HOV_degradataion[['District','Finance Target ($K)']], 
                            how = 'left', 
                            left_on = ['District'], 
                  right_on = ['District'])

In [103]:
dict_col_rename = {
    'Finance Target ($K)':'HOV Degradation Mitigation Projects (Years 6 & 7) HOV Degradation allocation ($K)'
}
df_LL_summary = df_LL_summary.rename(dict_col_rename, axis = 1)

In [104]:

#calculate HOV degradation cost for each district

temp = df_SHOPP_raw_data[(df_SHOPP_raw_data['Include 5-year POR?'] =='Yes') 
                  & (df_SHOPP_raw_data['HOV Degradation Project?'] =='Yes')].groupby(['District'])['Project Cost in RTL Year of POR'].sum().reset_index(name = 'HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)')

In [105]:
df_LL_summary = pd.merge(df_LL_summary, temp, how ='left', left_on = 'District', right_on = 'District')
df_LL_summary['HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)'].fillna(0, inplace=True)
df_LL_summary['HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)'] = round(df_LL_summary['HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)'],2)

In [106]:
def calc_HOV_degradation(df):
    if round(df['HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)'],0) >= round(df['HOV Degradation Mitigation Projects (Years 6 & 7) HOV Degradation allocation ($K)'],0):
        return 1
    else:
        return -1
    
df_LL_summary['HOV Degradation Mitigation Projects (Years 6 & 7) Equal or Greater Than Minimal Cost?'] = df_LL_summary.apply(calc_HOV_degradation, axis = 1)


In [107]:
df_LL_summary['Complete Streets Build New Performance (Years 6 & 7) % of Target'].fillna(0, inplace = True)
df_LL_summary['No of Projects'].fillna(0, inplace = True)

In [108]:
out_cols = ['District', '5-Y Funding Targets ($K)', 
        'Support Capital Ratio', 
       'Complete Streets Build New Performance (Years 6 & 7) % of Target',
       'Complete Streets Build New Performance (Years 6 & 7) Greater Than 40% Annual Performance Target?',
       'HOV Degradation Mitigation Projects (Years 6 & 7) HOV Degradation allocation ($K)',
       'HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)',
       'HOV Degradation Mitigation Projects (Years 6 & 7) Equal or Greater Than Minimal Cost?',
       'RTL In Use',  'Project Cost ($K)','No of Projects',
       'Percent of 5-Year Target Funding',
           ]


In [109]:
df_summary = df_LL_summary[out_cols]

df_summary = df_summary.append(df_5y_summary)

In [110]:
# calculate achieved performance

df_perf_ck = df_PPI_performace.groupby(['District','Performance Objective'])[['F2G Achieved', 'P2G Achieved',]].sum().reset_index()

df_perf_ck = pd.merge(df_perf_ck, df_finance_targets_uc[['District', 'Performance Objective', 'Unit of Measure','F2G Target', 'P2G Target',]],
                             how = 'left', left_on= ['District','Performance Objective'], right_on= ['District','Performance Objective'])



In [111]:
#modify the achieved performance and performance balance with different precision, use 0.1 only the safety columns, use 0 for all other columns

df_perf_ck['Precision'] = 1

df_perf_ck['Precision'].loc[(df_perf_ck['Performance Objective'].isin(['Proactive Safety','Reactive Safety']))] = 10

df_perf_ck['F2G Achieved'] = round(df_perf_ck['F2G Achieved'] * df_perf_ck['Precision'], 0) / df_perf_ck['Precision']
df_perf_ck['P2G Achieved'] = round(df_perf_ck['P2G Achieved'] * df_perf_ck['Precision'], 0) / df_perf_ck['Precision']


df_perf_ck['F2G Balance'] = -(df_perf_ck['F2G Achieved'] + df_perf_ck['F2G Target'])
df_perf_ck['P2G Balance'] = -(df_perf_ck['P2G Achieved'] + df_perf_ck['P2G Target'])


df_perf_ck['Meeting F2G Performance?'] = df_perf_ck['F2G Balance'].apply(lambda x: -1 if x < 0 else 1)
df_perf_ck['Meeting P2G Performance?'] = df_perf_ck['P2G Balance'].apply(lambda x: -1 if x < 0 else 1)

C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [112]:
df_perf_ck = df_perf_ck.sort_values(['District', 'Performance Objective'])

df_summary  = pd.concat([df_summary,df_perf_ck], join ='outer', ignore_index=True)

In [113]:
df_summary['Concatenate District +RTL'] = df_summary['District'].astype(str) + df_summary['RTL In Use']

df_summary['Concatenate District +objective'] = df_summary['District'].astype(str) + df_summary['Performance Objective']

df_summary['Date'] = TARGETDATE

<a id='Export_FaceSheets'></a>

# Export FactSheets


In [114]:
file_export_log = open(LOG_FILE, "a")  # append mode
file_export_log.write("#####{} \n".format(TARGETDATE))

17

In [115]:
out_cols = ['Date',
    'District', '5-Y Funding Targets ($K)',
    'Support Capital Ratio','Annual Target Funding ($K)', '5% Upper Limit ($M)','5% Lower Limit ($M)',
    'Complete Streets Build New Performance (Years 6 & 7) % of Target',
   'Complete Streets Build New Performance (Years 6 & 7) Greater Than 40% Annual Performance Target?',

   'HOV Degradation Mitigation Projects (Years 6 & 7) HOV Degradation allocation ($K)',
   'HOV Degradation Mitigation Projects (Years 6 & 7) Project Cost ($K)',
   'HOV Degradation Mitigation Projects (Years 6 & 7) Equal or Greater Than Minimal Cost?',
    
    'RTL In Use', 
            
    'Concatenate District +RTL',
    'Project Cost ($K)', 
    'No of Projects',
    'Balance ($K)',
    'Percent of 5-Year Target Funding',
    'Within 20% (+/-5%) Annual Target Funding?', 
    'PPI', 
    'Performance Objective',
    'Concatenate District +objective',
    'Unit of Measure',
    'F2G Target', 'P2G Target',    
    'F2G Achieved', 'P2G Achieved',  
    'F2G Balance', 'P2G Balance', 
    'Meeting F2G Performance?','Meeting P2G Performance?'       

]

df_out = df_summary[out_cols]

In [116]:
#rename df_out columns based on tableau convension 

rename_dict = {'RTL In Use': 'Fiscal Year', 
    'F2G Target':'Fair to Good SHSMP SHOPP Target',
    'P2G Target':'Poor to Good SHSMP SHOPP Target',    
    'F2G Achieved': 'Fair to Good Performance',
    'P2G Achieved': 'Poor to Good Performance',  
    'F2G Balance':'Fair to Good Performance Balance or Gap',
    'P2G Balance':'Poor to Good Performance Balance or Gap', 
}

df_out = df_out.rename(rename_dict, axis = 1)

In [117]:
export_data(df_out, 'factsheets', PROJECTBOOKCHECK_HTTPSEVER_FOLDER, LOG_FILE)

processing table: 480it [00:00, 12654.57it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing factsheets.hyper to Sandbox_ProjectBookCheck_Automation...


In [118]:
# #publish tableau datasource for factsheets
# filename = 'factsheets'

# try: 
#     df_out.to_csv('.\output\{}.csv'.format(filename), index= False)
#     shutil.copy('.\output\{}.csv'.format(filename), '{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename))
#     file_export_log.write("Succeeded: {} \n".format('{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename)))
# except:
#     file_export_log.write("Failed: {} \n".format('{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename)))


# hyper_name = 'factsheets.hyper'

# try: 
#     publish_datasource(df_out, hyper_name)
#     file_export_log.write("Succeeded: {} \n".format('{}'.format(hyper_name)))
# except:
#     file_export_log.write("Failed: {} \n".format('{}'.format(hyper_name)))


In [140]:
for c in ['Support Capital Ratio',
       'Project Cost In Use', 'New Target', 'F2G Target', 'P2G Target',
       'New Escalated UC ($K)', 'F2G Escalated UC ($K)',
       'P2G Escalated UC ($K)', 'F2G Achieved', 'P2G Achieved',
       'Earned Value (EV) ($K)', 'Effective Earned Value (EEV) ($K)',]:
    df_perf_raw_data_1[c].fillna(0, inplace=True)

In [135]:
# out_cols = ['District', 'AMT_ID','Section', 'Performance Objective', 
#             'F2G Achieved', 'P2G Achieved', 
#             'Concatenate District+Objective',
#             'Section', 
#             'Reservation Project?', 'Include 5-year POR?',
#             'Include in Pivot table?',
#        'RTL In Use', 'Support Capital Ratio', 'Project Cost In Use',
#         'New Escalated UC ($K)',   'F2G Escalated UC ($K)', 'P2G Escalated UC ($K)',
#             'New Target', 'F2G Target', 'P2G Target',
#             'Earned Value (EV) ($K)',
#        'Effective Earned Value (EEV) ($K)']
# df_out = df_perf_raw_data_1[out_cols]
df_perf_raw_data_1['PerformanceChange Date After Review'].fillna('NA', inplace = True)
df_perf_raw_data_1['Date'] = TARGETDATE

df_out = df_perf_raw_data_1[
    (df_perf_raw_data_1['Include 5-year POR?'] == 'Yes')]

In [129]:
export_data(df_out, 'performance', PROJECTBOOKCHECK_HTTPSEVER_FOLDER, LOG_FILE)

processing table: 528it [00:00, 5279.45it/s]

Table 'Extract' does not exist in extract performance.hyper, creating.


processing table: 7689it [00:01, 5216.07it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing performance.hyper to Sandbox_ProjectBookCheck_Automation...


In [121]:
# #publish tableau datasource for factsheets
# filename = 'performance'

# try: 
#     df_out.to_csv('.\output\{}.csv'.format(filename), index= False)
#     shutil.copy('.\output\{}.csv'.format(filename), '{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename))
#     file_export_log.write("Succeeded: {} \n".format('{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename)))
# except:
#     print('error')
#     file_export_log.write("Failed: {} \n".format('{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSEVER_FOLDER, filename)))


# hyper_name = 'performance.hyper'

# try: 
#     publish_datasource(df_out, hyper_name)
# #     print('success')
#     file_export_log.write("Succeeded: {} \n".format('{}'.format(hyper_name)))
# except:
#     print('error')
#     file_export_log.write("Failed: {} \n".format('{}'.format(hyper_name)))


<a id='FinalCleanUp'></a>
## Final Clean Up

In [122]:
file_export_log.close()

In [123]:


#clean up tableau publishing log file

import os
import glob
# get a recursive list of file paths that matches pattern
fileList = glob.glob('./*.log')
# Iterate over the list of filepaths & remove each file.
for filePath in fileList:
    try:
        os.remove(filePath)
    except OSError:
        print("Error while deleting file")


In [124]:
end_time =  time.time()
elapsed = end_time - start_time
print('time elapsed : {} seconds'.format(elapsed))

time elapsed : 20.66102433204651 seconds
